In [1]:
import evaluate
from evaluate import load
import tensorflow_hub as hub
from scipy.spatial import distance
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import pandas as pd

c:\Users\adrianhf\anaconda3\envs\master\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\adrianhf\anaconda3\envs\master\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


c:\Users\adrianhf\anaconda3\envs\master\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\adrianhf\anaconda3\envs\master\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
bertscore = load("bertscore")
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')

In [3]:
references = load_dataset('csv', data_files=r'C:\Users\adrianhf\Documents\test\Master\data\synthetic_data\question_with_answers.csv', split="train[:10]")
predictions = load_dataset('csv', data_files=r'C:\Users\adrianhf\Documents\test\Master\data\Results\Chroma_answers_from_model.csv')

Generating train split: 10 examples [00:00, 70.60 examples/s]


In [4]:
references = references["Answer"]
predictions = predictions["train"]["Text"]

In [5]:
bert_score = bertscore.compute(predictions=predictions, references=references, lang="nb")
bleu_score = bleu.compute(predictions=predictions, references=references, max_order=2)
rouge_score = rouge.compute(predictions=predictions, references=references)

c:\Users\adrianhf\anaconda3\envs\master\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [6]:
avg_precision = sum(bert_score['precision']) / len(bert_score['precision'])
avg_recall = sum(bert_score['recall']) / len(bert_score['recall'])
avg_f1 = sum(bert_score['f1']) / len(bert_score['f1'])

In [7]:
## SAS encoder score
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print("module %s loaded" % module_url)
 
 
def embed(input):
    return model(input)
list_of_similarity_scores = []
for i in range(len(predictions)):
    similarity_score = 1-distance.cosine(embed([predictions[i]])[0, :],embed([references[i]])[0, :])
    list_of_similarity_scores.append(similarity_score)
    print(f'\nPrediction: {predictions[i]}\nReference: {references[i]}\nSimilarity Score = {similarity_score} ')
average_score = sum(list_of_similarity_scores) / len(list_of_similarity_scores)
print("Average similarity score:", average_score)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded

Prediction: 26. juni 1984
Reference: Datoen for vedtaket av Kommunedelplan for sentrum av bystyret var 26.8.2021.
Similarity Score = 0.011732097015851983 

Prediction: 

Reference: Hovedintensjonene i planen beskrevet i dokumentet inkluderer å videreutvikle Kristiansund sentrum som et attraktivt og framtidsrettet regionbysentrum med et pulserende folkeliv. Dette inkluderer positiv utvikling av handels- og servicevirksomhet, god parkerings- og trafikksituasjon, hensyn til gjenreisningsarkitekturens formuttrykk, god framkommelighet og trygghet for alle grupper, et mangfold av kulturaktiviteter, ønske om flere boligprosjekter i sentrum, stimulering til flere arbeids- og skoleplasser i sentrum, og utvikling av sentrum til en regional attraktiv, tett og urban bolig-, handels- og kulturby med gode bymessige kvaliteter.
Similarity Score = 0.011246906593441963 

Prediction: 
1. Å utnytte potensialet i eksisterende bysentralis

In [8]:
## SAS transformer score
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

def SAS(preds, refs):
    similarities = []
    embeddings_preds = model.encode(preds)
    embeddings_refs = model.encode(refs)
    for i in range(len(embeddings_preds)):
        similarity = util.pytorch_cos_sim(embeddings_preds[i], embeddings_refs[i])
        similarities.append(similarity[0][0].item())
    average_similarity_score = sum(similarities) / len(similarities)
    return average_similarity_score

In [9]:
print("BLEU SCORES")
print(bleu_score)
print("ROUGE SCORES")
print(rouge_score)
print("BERT SCORES")
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1 Score:", avg_f1)
print("Average SAS encoder Score:", average_score)
print("Average SAS transformer Score:", SAS(predictions, references))

BLEU SCORES
{'bleu': 0.09433699363819179, 'precisions': [0.27146814404432135, 0.07954545454545454], 'brevity_penalty': 0.6419702625382296, 'length_ratio': 0.6928982725527831, 'translation_length': 361, 'reference_length': 521}
ROUGE SCORES
{'rouge1': 0.154166030048383, 'rouge2': 0.04602619426148837, 'rougeL': 0.12694805194805198, 'rougeLsum': 0.1309552457346575}
BERT SCORES
Average Precision: 0.6244073987007142
Average Recall: 0.5748213291168213
Average F1 Score: 0.5956018567085266
Average SAS encoder Score: 0.4121939039849362
Average SAS transformer Score: 0.5537798725068569


In [10]:
data = {
    "Metric": ["BLEU Score", "ROUGE Score", "Average Precision", "Average Recall", "Average F1 Score", "Average SAS encoder Score", "Average SAS transformer Score"],
    "Score": [bleu_score, rouge_score, avg_precision, avg_recall, avg_f1, average_score, SAS(predictions, references)]
}

# Create DataFrame
df = pd.DataFrame(data)

# Specify the file path
file_path = "C:\\Users\\adrianhf\\Documents\\test\\Master\\data\\Evaluation_scores\\Chroma_evaluation_scores.csv"

# Write DataFrame to CSV
df.to_csv(file_path, index=False)

print("Data has been written to", file_path)

Data has been written to C:\Users\adrianhf\Documents\test\Master\data\Evaluation_scores\Chroma_evaluation_scores.csv
